In [1]:
# Install the sentence-transformers library for generating text embeddings
!pip install -q sentence-transformers

# Mount Google Drive and load data
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd

# Define file paths
train_path = '/content/drive/MyDrive/EECS4412/Project/train_yelp_60k.csv'
test_path = '/content/drive/MyDrive/EECS4412/Project/test_yelp_60k.csv'

# Load training and test datasets
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

# Clean column names
train_df.columns = train_df.columns.str.strip()
test_df.columns = test_df.columns.str.strip()

# Preview first few rows
train_df.head()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 88.0 MB/s eta 0:00:00
Mounted at /content/drive


,Text,Class,ID
0,Chef Kevin Sousa's 2018 award winning restaur...,positive,727658
1,This place has got potential. I did quite enjo...,positive,5407165
2,I was really excited to try this place as they...,neutral,2753394
3,Heyyyyy waffle sandwiches! Who doesn't love a ...,positive,1735407
4,Goods and Provisions has a lovely atmosphere ...,negative,1388216


In [2]:
# Check class distribution
print("Class distribution in training set:")
print(train_df['Class'].value_counts())

# Check for missing values
print("\nMissing values in training data:")
print(train_df.isnull().sum())

# Show a sample review
print("\nSample text from training data:")
print(train_df['Text'].iloc[0])


Class distribution in training set:
Class
positive    39328
negative    14028
neutral      6644
Name: count, dtype: int64

Missing values in training data:
Text     0
Class    0
ID       0
dtype: int64

Sample text from training data:
Chef Kevin Sousa's  2018 award winning restaurant sits in the middle of an old neighborhood in Braddock  PA in what was once a Chevy dealership. The eclectic farm to table menu is laid out differently than most restaurants that we've been to.Parking is free either on the street or in what was once known as the dealership's parking lot. When we arrived we were seated promptly and a very friendly and helpful waiter guided us through the menu and answered any questions we had. He explained to us that in creating the menu they deviated from the traditional style of appetizers  salads  mains  etc to more of a four course selection where you can pick and choose whatever you wanted. The first page is the entire food menu. The pages that follow it consists of you

In [3]:
from sentence_transformers import SentenceTransformer

# Load pre-trained sentence embedding model
embedder = SentenceTransformer('all-MiniLM-L6-v2')

# Convert training reviews to a list
train_texts = train_df['Text'].tolist()

# Generate sentence embeddings (can take time on large datasets)
X_train = embedder.encode(train_texts, show_progress_bar=True)
y_train = train_df['Class']

print("X_train shape:", X_train.shape)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1875 [00:00<?, ?it/s]

X_train shape: (60000, 384)


In [4]:
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier

# Define classifiers to evaluate
models = {
    "LogisticRegression": LogisticRegression(max_iter=1000),
    "LinearSVC": LinearSVC(max_iter=2000),
    "RandomForest": RandomForestClassifier(n_estimators=100)
}

# 5-fold stratified cross-validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Evaluate models and report accuracy
for name, model in models.items():
    scores = cross_val_score(model, X_train, y_train, cv=cv, scoring='accuracy', n_jobs=-1)
    print(f"{name} CV Accuracy: {scores.mean():.4f} ± {scores.std():.4f}")


LogisticRegression CV Accuracy: 0.8258 ± 0.0014
LinearSVC CV Accuracy: 0.8262 ± 0.0014
RandomForest CV Accuracy: 0.7834 ± 0.0022


In [5]:
# Based on CV results, LinearSVC performed best, so we choose it.
final_clf = LinearSVC(max_iter=2000)
final_clf.fit(X_train, y_train)


LinearSVC(max_iter=2000)

In [6]:
# Generate embeddings for the test set
test_texts = test_df['Text'].tolist()
X_test = embedder.encode(test_texts, show_progress_bar=True)

# Predict labels for test data
test_preds = final_clf.predict(X_test)

# Save predictions to 'prediction2.csv' in the required format
import csv

with open('prediction2.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['ID', 'CLASS'])  # Write header
    for review_id, pred_class in zip(test_df['ID'], test_preds):
        writer.writerow([review_id, pred_class])


Batches:   0%|          | 0/1875 [00:00<?, ?it/s]